# Download the Dataset

In [1]:
!wget https://philharmonia-assets.s3-eu-west-1.amazonaws.com/uploads/2020/02/12112005/all-samples.zip

--2020-06-08 08:31:34--  https://philharmonia-assets.s3-eu-west-1.amazonaws.com/uploads/2020/02/12112005/all-samples.zip
Resolving philharmonia-assets.s3-eu-west-1.amazonaws.com (philharmonia-assets.s3-eu-west-1.amazonaws.com)... 52.218.56.176
Connecting to philharmonia-assets.s3-eu-west-1.amazonaws.com (philharmonia-assets.s3-eu-west-1.amazonaws.com)|52.218.56.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 260849422 (249M) [application/zip]
Saving to: ‘all-samples.zip’

all-samples.zip     100%[===================>] 248.76M  30.8MB/s    in 8.7s    

2020-06-08 08:31:44 (28.4 MB/s) - ‘all-samples.zip’ saved [260849422/260849422]



# Extract zip files

In [0]:
from zipfile import ZipFile
with ZipFile('all-samples.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [0]:
import os
dir_path='./all-samples'

In [0]:
for path, dir_list, file_list in os.walk(dir_path):
    for file_name in file_list:
        if file_name.endswith(".zip"):
            abs_file_path = os.path.join(path, file_name)

            parent_path = os.path.split(abs_file_path)[0]
            output_folder_name = os.path.splitext(abs_file_path)[0]
            output_path = os.path.join(parent_path, output_folder_name)

            zip_obj = ZipFile(abs_file_path, 'r')
            zip_obj.extractall(output_path)
            zip_obj.close()

# Start Trials

In [0]:
path_dataset='./all-samples/all-samples/'
import fnmatch

In [12]:
instruments = ['banjo','bass-clarinet','bassoon', 'cello','clarinet','contrabassoon',
               'english-horn','double-bass','flute','french-horn','guitar','mandolin',
               'oboe','percussion','saxophone','trombone','trumpet','tuba','viola','violin']
classes = []

counter=1
for root, dirnames, filenames in os.walk(path_dataset):
    for i,filename in enumerate(filenames):
      print ("Get %d = %s"%(counter, filename))
      counter+=1
      if 'percussion' in root:
        classes.append('percussion')
      else:
        for instrument in instruments:
          if fnmatch.fnmatchcase(filename, instrument+'*'):
            classes.append(instrument)

Streaming output truncated to the last 5000 lines.
Get 8685 = bassoon_F2_phrase_forte_fluttertonguing.mp3
Get 8686 = bassoon_As1_15_forte_normal.mp3
Get 8687 = bassoon_Ds2_05_mezzo-piano_normal.mp3
Get 8688 = bassoon_G2_phrase_forte_fluttertonguing.mp3
Get 8689 = bassoon_Cs3_025_mezzo-piano_normal.mp3
Get 8690 = bassoon_Ds2_15_piano_normal.mp3
Get 8691 = bassoon_A4_05_forte_normal.mp3
Get 8692 = bassoon_A4_1_forte_normal.mp3
Get 8693 = bassoon_C2_1_fortissimo_normal.mp3
Get 8694 = bassoon_C4_025_forte_normal.mp3
Get 8695 = bassoon_A4_15_piano_normal.mp3
Get 8696 = bassoon_Gs4_05_fortissimo_normal.mp3
Get 8697 = bassoon_Ds5_05_fortissimo_normal.mp3
Get 8698 = bassoon_D4_15_piano_normal.mp3
Get 8699 = bassoon_As3_long_forte_major-trill.mp3
Get 8700 = bassoon_Gs3_05_piano_normal.mp3
Get 8701 = bassoon_D4_1_forte_normal.mp3
Get 8702 = bassoon_Ds3_1_forte_normal.mp3
Get 8703 = bassoon_G3_025_mezzo-piano_normal.mp3
Get 8704 = bassoon_F2_025_forte_normal.mp3
Get 8705 = bassoon_Gs3_025_forte_n

In [7]:
import pandas as pd
ds_Classes = pd.Series(classes)
ds_Classes.value_counts()

violin           1502
viola             974
tuba              972
bass-clarinet     944
cello             889
flute             878
double-bass       852
clarinet          846
trombone          831
saxophone         733
bassoon           720
contrabassoon     710
english-horn      691
french-horn       652
oboe              596
trumpet           485
percussion        148
guitar            106
mandolin           80
banjo              74
dtype: int64

Are the numbers corect? You can check that by comparing to the number for files in each folder using the properties in Windows Explorer.

How can you change the code now to include the duration?